<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado en 2017-2, 2018-1, 2018-2, 2019-1, 2019-2, 2020-1, 2020-2 y 2023-1 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos

1. [Clases de Excepciones](#Clases-de-Excepciones)
2. [Excepciones personalizadas](#Excepciones-personalizadas)

# Clases de Excepciones

En Python, todas las excepciones heredan de `BaseException`. A partir de ella existen tres tipos de excepciones: **`SystemExit`**, **`KeyboardInterrupt`**, y **`Exception`**. Todas las excepciones generadas por errores durante la ejecución de un programa son subclases de `Exception`, tal como se muestra en el siguiente diagrama:

![](img/jerarquia-excepciones.png)

Esto quiere decir, que si se usa `Exception` para manejar errores (`except Exception`), esto capturará cualquier error que sea instancia de una subclase de `Exception`. De esta forma, el tratamiento es general y no específico a un tipo de error en especial. En general, es recomendable actuar de forma **selectiva** sobre un tipo determinado de excepciones (`IOError`, `AtributeError`, `ValueError`, etc.). Si bien es posible manejar cualquier tipo de excepción capturando `Exception`, esto se considera una **mala práctica**, pues capturar una excepción sin saber su naturaleza, puede causar comportamientos inesperados en el programa.

In [1]:
def dividir(num, den):
    if not (isinstance(num, int) and isinstance(den, int)):
        raise TypeError("Error de tipo en numerador o denominador. :'(")

    if num < 0 or den < 0:
        raise ValueError("Hay un valor negativo entre numerador y denominador >:(")

    return float(num) / float(den)

In [2]:
try:
    print(dividir(1, "1"))
except Exception as err:
    # Este bloque opera para todos los tipos de excepciones que hereden de Exception.
    print(f"Error: {err}")
    print("Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... ")

Error: Error de tipo en numerador o denominador. :'(
Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... 


In [3]:
try:
    print(dividir(1, -2))
except Exception as err:
    # Este bloque opera para todos los tipos de excepciones que hereden de Exception.
    print(f"Error: {err}")
    print("Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... ")

Error: Hay un valor negativo entre numerador y denominador >:(
Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... 


In [4]:
try:
    print(dividir(4, 0))
except Exception as err:
    # Este bloque opera para todos los tipos de excepciones que hereden de Exception.
    print(f"Error: {err}")
    print("Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... ")

Error: float division by zero
Revise los datos de entrada. Algo pasó ahí, pero no sé qué tipo específico de Exception ... 


# Excepciones personalizadas

Podemos crear nuestros propios tipos de excepciones. Para ello debemos heredar desde la clase `Exception`. Podemos modificar el comportamiento heredado sobrescribiendo los métodos que tiene implementada esta clase.

In [5]:
class Excepcion1(Exception):

    # Al no sobrescribir nada, hereda todo sin modificaciones.
    pass


class Excepcion2(Exception):

    def __init__(self, a, b):
        # Sobrescribimos el __init__ para cambiar el ingreso de los parámetros.
        super().__init__(f"Alguno de los valores {a} o {b} no es entero")


def dividir(num, den):
    # Por ejemplo, redefiniremos las excepciones que
    # utilizamos en los ejemplos anteriores.
    if not (isinstance(num, int) and isinstance(den, int)):
        raise Excepcion2(num, den)

    if num < 0 or den < 0:
        raise Excepcion1("Los valores son negativos")

    return float(num) / float(den)

In [6]:
# Este ejempo lanza una Excepcion1.
try:
    print(dividir(4, -3))

except Excepcion1 as err:
    # Este bloque opera para la Excepcion1.
    print(f"Error: {err}")

except Excepcion2 as err:
    # Este bloque opera para Excepcion2 cuando los datos no son enteros.
    print(f"Error: {err}")

Error: Los valores son negativos


In [7]:
# Este ejemplo lanza una Excepcion2.
try:
    print(dividir(4.4, -3))

except Excepcion1 as err:
    # Este bloque opera para la Excepcion1.
    print(f"Error: {err}")

except Excepcion2 as err:
    # Este bloque opera para Excepcion2 cuando los datos no son enteros.
    print(f"Error: {err}")

Error: Alguno de los valores 4.4 o -3 no es entero


Podemos definir comportamientos personalizados para las excepciones que creamos, por ejemplo, agregar métodos que nos permitan recuperar información de la excepción.

In [8]:
class ErrorTransaccion(Exception):

    def __init__(self, fondos, gasto):
        super().__init__(f"El dinero en la billetera no alcanza para pagar ${gasto}")
        self.fondos = fondos
        self.gasto = gasto

    def exceso(self):
        return self.gasto - self.fondos


class Billetera:

    def __init__(self, dinero):
        self.fondos = dinero

    def pagar(self, gasto):
        if self.fondos - gasto < 0:
            raise ErrorTransaccion(self.fondos, gasto)
        self.fondos -= gasto


b = Billetera(1000)

try:
    b.pagar(1500)

except ErrorTransaccion as err:
    print(f"Error: {err}. Hay un exceso de gastos de ${err.exceso()}.")

Error: El dinero en la billetera no alcanza para pagar $1500. Hay un exceso de gastos de $500.


## Un ejemplo: La lista del curso

Este ejemplo desea modelar la siguiente situación: se desean inscribir estudiantes en distintos cursos universitarios, para lo cual se cuentan con listas de las peticiones de varios estudiantes por curso. El problema es que estas listas fueron afectadas por *hackers* que poblaron de información falsa, específicamente, hay estudiantes con número de identificación duplicados. Ya que no hay forma de identificar los falsos de los verdaderos, se inscribirán solamente quienes tengan un identificador nuevo, y los duplicados se almacenarán de alguna forma, para futura referencia.

La siguiente solución utiliza una excepción personalizada para llevar el registro de los duplicados encontrados en la lista. Aprovecha el uso de atributos de clase para mantener un registro general de todos los duplicados, y atributos de instancia para obtener información.

**(Abre paréntesis...**

Al usar clases, normalmente definimos atributos para las instancias mediante el uso de `self.atributo = valor`. Esto genera la creación de un **atributo de instancia**, cuyo valor es accesible desde cualquier punto de la instancia mediante `self`. Si definimos un atributo al nivel de los métodos, se le conoce como **atributo de clase** y es accesible por todas las instancias mediante el nombre de la clase: `Clase.atributo`. También, es posible acceder mediante `self` en una instancia, siempre y cuando no tenga un atributo de instancia del mismo nombre definido. Cuando se busca un valor de atributo mediante `self.atributo`, el orden de búsqueda es:
1. Buscar en la instancia. (atributo de instancia)
2. Si no existe, buscar en la clase. (atributo de clase)
3. Si no existe, levantar un error (`AttributeError`).

**...cierra paréntesis.)**

In [9]:
from collections import namedtuple, defaultdict


# Modelamos les estudiantes como entidades simples.
Estudiante = namedtuple("Estudiante", ["numero", "nombre", "apellido"])


class ErrorRepeticionEstudiante(Exception):

    # Se mantiene un diccionario como atributo de clase para almacenar todos los duplicados.
    estudiantes_repetidos = defaultdict(list)

    def __init__(self, estudiante):
        # Se almacena le alumne que lanzó la excepción.
        self.estudiante = estudiante
        numero = estudiante.numero
        # Se almacena el duplicado.
        self.estudiantes_repetidos[numero].append(estudiante)
        super().__init__(f"¡Error de estudiante repetido de número: {estudiante}")

    # Creamos property de instancia que accede a todos los duplicados del mismo número.
    @property
    def repetidos(self):
        return self.estudiantes_repetidos[self.estudiante.numero]

    # Creamos property de instancia que calcula la cantidad de duplicados del mismo número.
    @property
    def cantidad(self):
        return len(self.estudiantes_repetidos[self.estudiante.numero])


# Modelamos el curso con una estructura simple para almacenar sus estudiantes
class Curso:

    def __init__(self, nombre):
        self.nombre = nombre
        self.estudiantes = dict()

    # Método encargado de inscribir nuevo estudiante, en caso de encontrar duplicado, levanta un error.
    def inscribir(self, estudiante):
        if estudiante.numero in self.estudiantes:
            raise ErrorRepeticionEstudiante(estudiante)
        else:
            self.estudiantes[estudiante.numero] = estudiante
            print(f"✅ Estudiante de número {estudiante.numero} inscrito exitosamente.")

In [10]:
estudiantes_no_inscritos = [
    Estudiante('15633459', 'Juan', 'Hernández'),
    Estudiante('1663525J', 'Belén', 'Pinto'),
    Estudiante('17632451', 'Ariel', 'Gonzalez'),
    Estudiante('15633459', 'Fernanda', 'Errazuriz'),
    Estudiante('1563001J', 'Javiera', 'Martínez'),
    Estudiante('1663525J', 'Benjamín', 'Valdivieso'),
    Estudiante('15633459', 'Cristian', 'Soto'),
]

iic2233 = Curso('IIC2233')

for estudiante in estudiantes_no_inscritos:
    try:
        iic2233.inscribir(estudiante)
    except ErrorRepeticionEstudiante as error:
        print(f"❌ Estudiante repetido de número {estudiante.numero} encontrado.")
        print(f"Ya van {error.cantidad} estudiantes repetidos con el número {estudiante.numero}:")
        print("\t" + ", ".join([f"{estudiante.nombre} {estudiante.apellido}" for estudiante in error.repetidos]))

✅ Estudiante de número 15633459 inscrito exitosamente.
✅ Estudiante de número 1663525J inscrito exitosamente.
✅ Estudiante de número 17632451 inscrito exitosamente.
❌ Estudiante repetido de número 15633459 encontrado.
Ya van 1 estudiantes repetidos con el número 15633459:
	Fernanda Errazuriz
✅ Estudiante de número 1563001J inscrito exitosamente.
❌ Estudiante repetido de número 1663525J encontrado.
Ya van 1 estudiantes repetidos con el número 1663525J:
	Benjamín Valdivieso
❌ Estudiante repetido de número 15633459 encontrado.
Ya van 2 estudiantes repetidos con el número 15633459:
	Fernanda Errazuriz, Cristian Soto


**Revisa los ejercicios propuestos de la sección 3 para poner en práctica la creación de excepciones personalizadas.**